## Quantitative Value Strategy 
### Obtaining Data

In [3]:
!pip install yahoo_fin
!pip install xlsxwriter

In [6]:
from yahoo_fin.stock_info import tickers_nasdaq, get_stats_valuation, get_live_price
import pandas as pd
import numpy as np
import math
from scipy import stats
from statistics import mean

In [ ]:
tickers = tickers_nasdaq() #Getting Tickers of all Stocks Listed on Nasdaq
dataset = pd.DataFrame(columns = ["Ticker", "Price/Earnings", "Price/Sales (TTM)", "Price/Book (MRQ)", "Enterprise Value/Revenue",	"Enterprise Value/EBITDA", "Price"])

In [ ]:
for ticker in tickers:
  try:
    data = get_stats_valuation(ticker)
    price = get_live_price(ticker)
    dataset.loc[len(dataset.index)] = [ticker, data[1][5], data[1][4], data[1][6], data[1][7], data[1][8], price]
  except:
    pass

In [ ]:
dataset.isna().sum()

In [ ]:
data = dataset.dropna(subset = ["Ticker", "Price/Earnings", "Price/Sales (TTM)", "Price/Book (MRQ)", "Enterprise Value/Revenue", "Enterprise Value/EBITDA"])
# data

In [ ]:
data.to_csv("Valuation_Measures_Data.csv", index = False)

### Analysis


In [ ]:
data = dataset.reindex(columns=[*data.columns.tolist(), 'P/E Percentile', 'P/S Percentile', 'P/B Percentile', 'EV/R Percentile', 'EV/EBITDA Percentile', 'RV Score', "Number of Shares to Buy"], fill_value=0)
data

,Ticker,Price/Earnings,Price/Sales (TTM),Price/Book (MRQ),Enterprise Value/Revenue,Enterprise Value/EBITDA,Price,P/E Percentile,P/S Percentile,P/B Percentile,EV/R Percentile,EV/EBITDA Percentile,RV Score,Number of Shares to Buy
1,AADI,1.89,6.2,0.98,-0.23,0.08,4.84,0,0,0,0,0,0,0
3,AAON,30.74,4.37,6.87,4.75,24.57,56.87,0,0,0,0,0,0,0
4,AAPL,28.85,7.13,44.6,7.12,21.56,171.21,0,0,0,0,0,0,0
5,ABCL,19.92,9.79,1.14,3.58,-48.93,4.60,0,0,0,0,0,0,0
6,ABCM,1.85k,11.33,5.92,11.33,69.19,22.63,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,ZEUS,14.88,0.26,1.09,0.37,9.04,56.21,0,0,0,0,0,0,0
2356,ZI,50.9,5.31,2.7,5.83,17.31,16.40,0,0,0,0,0,0,0
2359,ZJYL,40.27,6.54,5.55,5.95,36.94,23.40,0,0,0,0,0,0,0
2360,ZKIN,11.93,0.21,0.26,0.33,-13.97,0.70,0,0,0,0,0,0,0


### Creating Robust Value Metric

In [ ]:
metrics = {
            'Price/Earnings': 'P/E Percentile',
            'Price/Sales (TTM)': 'P/S Percentile',
            'Price/Book (MRQ)':'P/B Percentile',
            'Enterprise Value/Revenue':'EV/R Percentile',
            'Enterprise Value/EBITDA':'EV/EBITDA Percentile',
}

for row in data.index:
    for metric in metrics.keys():
        data.loc[row, metrics[metric]] = stats.percentileofscore(data[metric], data.loc[row, metric]) / 100

data

,Ticker,Price/Earnings,Price/Sales (TTM),Price/Book (MRQ),Enterprise Value/Revenue,Enterprise Value/EBITDA,Price,P/E Percentile,P/S Percentile,P/B Percentile,EV/R Percentile,EV/EBITDA Percentile,RV Score,Number of Shares to Buy
1361,MNTK,94.8,7.58,5.98,7.62,31.75,9.11,99.148211,94.974446,92.419080,96.337308,68.356048,0.902470,0
1986,STAA,80.94,6.55,5.55,5.89,57.97,40.18,94.463373,92.589438,89.991482,92.206133,81.005111,0.900511,0
394,CDMO,9.41,4.24,3.31,5.29,97.26,9.44,97.274276,83.304940,75.766610,89.224872,99.744463,0.890630,0
1445,NFLX,40.35,5.33,7.36,5.49,8.66,377.60,70.783646,88.756388,96.592845,90.587734,94.207836,0.881857,0
1427,NBIX,64.42,6.95,6.04,6.27,40.8,112.50,85.945486,93.611584,92.759796,93.441227,73.850085,0.879216,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,TBLT,0.01,0.03,0.41,0.1,-0.52,0.19,0.212947,0.638842,7.155026,8.177172,2.044293,0.036457,0
1451,NISN,0.6,0.05,0.06,-0.23,-2.18,2.39,2.683135,1.320273,0.383305,1.277683,7.921635,0.027172,0
1220,LIZI,1.23,0.06,0.29,-0.03,-0.7,3.86,4.429302,1.703578,4.003407,0.170358,2.896082,0.026405,0
1028,IMPP,0.32,0.12,0.12,-0.29,-0.59,1.58,1.703578,4.045997,1.405451,1.448041,2.470187,0.022147,0


In [ ]:
for row in data.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(data.loc[row, metrics[metric]])
    data.loc[row, 'RV Score'] = mean(value_percentiles)

data

,Ticker,Price/Earnings,Price/Sales (TTM),Price/Book (MRQ),Enterprise Value/Revenue,Enterprise Value/EBITDA,Price,P/E Percentile,P/S Percentile,P/B Percentile,EV/R Percentile,EV/EBITDA Percentile,RV Score,Number of Shares to Buy
1361,MNTK,94.8,7.58,5.98,7.62,31.75,9.11,99.148211,94.974446,92.419080,96.337308,68.356048,90.247019,0
1986,STAA,80.94,6.55,5.55,5.89,57.97,40.18,94.463373,92.589438,89.991482,92.206133,81.005111,90.051107,0
394,CDMO,9.41,4.24,3.31,5.29,97.26,9.44,97.274276,83.304940,75.766610,89.224872,99.744463,89.063032,0
1445,NFLX,40.35,5.33,7.36,5.49,8.66,377.60,70.783646,88.756388,96.592845,90.587734,94.207836,88.185690,0
1427,NBIX,64.42,6.95,6.04,6.27,40.8,112.50,85.945486,93.611584,92.759796,93.441227,73.850085,87.921635,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,TBLT,0.01,0.03,0.41,0.1,-0.52,0.19,0.212947,0.638842,7.155026,8.177172,2.044293,3.645656,0
1451,NISN,0.6,0.05,0.06,-0.23,-2.18,2.39,2.683135,1.320273,0.383305,1.277683,7.921635,2.717206,0
1220,LIZI,1.23,0.06,0.29,-0.03,-0.7,3.86,4.429302,1.703578,4.003407,0.170358,2.896082,2.640545,0
1028,IMPP,0.32,0.12,0.12,-0.29,-0.59,1.58,1.703578,4.045997,1.405451,1.448041,2.470187,2.214651,0


### Defining Portfolio

In [ ]:
def portfolio_input():
    global portfolio_value, number_of_stocks
    portfolio_value = input("Enter the value of your portfolio:")
    number_of_stocks = input("Enter the number of stocks you want to invest in:")

    try:
        value_1 = float(portfolio_value)
        value_2 = float(number_of_stocks)

    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_value = input("Enter the value of your portfolio:")
        number_of_stocks = input("Enter the number of stocks you want to invest in:")

In [ ]:
portfolio_input()

Enter the value of your portfolio:1000
Enter the number of stocks you want to invest in:25


In [ ]:
data.sort_values(by = 'RV Score', inplace = True)
portfolio = data[:int(number_of_stocks)]
portfolio.reset_index(drop = True, inplace = True)
portfolio

,Ticker,Price/Earnings,Price/Sales (TTM),Price/Book (MRQ),Enterprise Value/Revenue,Enterprise Value/EBITDA,Price,P/E Percentile,P/S Percentile,P/B Percentile,EV/R Percentile,EV/EBITDA Percentile,RV Score,Number of Shares to Buy
0,TANH,0.46,0.03,0.04,-0.09,-0.69,1.35,2.129472,0.638842,0.170358,0.724020,2.810903,1.294719,74
1,IMPP,0.32,0.12,0.12,-0.29,-0.59,1.58,1.703578,4.045997,1.405451,1.448041,2.470187,2.214651,63
2,LIZI,1.23,0.06,0.29,-0.03,-0.7,3.86,4.429302,1.703578,4.003407,0.170358,2.896082,2.640545,25
3,NISN,0.6,0.05,0.06,-0.23,-2.18,2.39,2.683135,1.320273,0.383305,1.277683,7.921635,2.717206,41
4,TBLT,0.01,0.03,0.41,0.1,-0.52,0.19,0.212947,0.638842,7.155026,8.177172,2.044293,3.645656,526
5,FAMI,0.1,0.04,0.05,-0.43,-9.01,1.60,0.851789,1.022147,0.255537,1.959114,14.821124,3.781942,62
6,GSMG,1.06,0.17,0.15,-0.74,-4.56,0.39,3.747871,5.877342,1.959114,2.470187,11.584327,5.127768,256
7,WGS,1.96,0.26,0.31,0.05,-0.03,3.64,6.686542,9.752981,4.557070,6.856899,0.255537,5.621806,27
8,NUWE,0.56,0.13,0.25,-0.63,0.73,1.43,2.555366,4.471891,3.321976,2.257240,18.867121,6.294719,69
9,METX,1.33,0.11,0.15,0.04,0.08,3.55,4.684838,3.662692,1.959114,6.345826,15.800681,6.490630,11267605


In [ ]:
position_size = float(portfolio_value) / int(number_of_stocks)

for i in range(0, len(portfolio['Ticker'])):
    portfolio.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / portfolio['Price'][i])
portfolio

,Ticker,Price/Earnings,Price/Sales (TTM),Price/Book (MRQ),Enterprise Value/Revenue,Enterprise Value/EBITDA,Price,P/E Percentile,P/S Percentile,P/B Percentile,EV/R Percentile,EV/EBITDA Percentile,RV Score,Number of Shares to Buy
0,TANH,0.46,0.03,0.04,-0.09,-0.69,1.35,2.129472,0.638842,0.170358,0.724020,2.810903,1.294719,29.0
1,IMPP,0.32,0.12,0.12,-0.29,-0.59,1.58,1.703578,4.045997,1.405451,1.448041,2.470187,2.214651,25.0
2,LIZI,1.23,0.06,0.29,-0.03,-0.7,3.86,4.429302,1.703578,4.003407,0.170358,2.896082,2.640545,10.0
3,NISN,0.6,0.05,0.06,-0.23,-2.18,2.39,2.683135,1.320273,0.383305,1.277683,7.921635,2.717206,16.0
4,TBLT,0.01,0.03,0.41,0.1,-0.52,0.19,0.212947,0.638842,7.155026,8.177172,2.044293,3.645656,210.0
5,FAMI,0.1,0.04,0.05,-0.43,-9.01,1.60,0.851789,1.022147,0.255537,1.959114,14.821124,3.781942,25.0
6,GSMG,1.06,0.17,0.15,-0.74,-4.56,0.39,3.747871,5.877342,1.959114,2.470187,11.584327,5.127768,102.0
7,WGS,1.96,0.26,0.31,0.05,-0.03,3.64,6.686542,9.752981,4.557070,6.856899,0.255537,5.621806,10.0
8,NUWE,0.56,0.13,0.25,-0.63,0.73,1.43,2.555366,4.471891,3.321976,2.257240,18.867121,6.294719,27.0
9,METX,1.33,0.11,0.15,0.04,0.08,3.55,4.684838,3.662692,1.959114,6.345826,15.800681,6.490630,11.0


### Exporting Output to Excel File

In [ ]:
values = {"Ticker": portfolio["Ticker"], "Price": portfolio["Price"], "Number of Shares to Buy": portfolio["Number of Shares to Buy"]}
final_data = pd.DataFrame(data = values)
final_data.to_excel("Value_Strategy.xlsx", sheet_name='Value Strategy', index = False)